In [1]:
from pnet import pnet_loader, Pnet, PnetOG, ReactomeNetworkOG
from util import util, sankey_diag

import torch
import seaborn as sns
import pandas as pd
import numpy as np
import random
import pickle
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import os
import torch.nn.functional as F
import torch.nn as nn

%load_ext autoreload
%autoreload 2

/root/miniconda3/envs/pnet/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generate small test dataset

In [2]:
# rna_ext_val = pd.read_csv('/mnt/disks/pancan/data/mel_dfci_2019/data_RNA_Seq_expression_tpm_all_sample_Zscores.txt',
#                           delimiter='\t').set_index('Hugo_Symbol').T.drop('Entrez_Gene_Id').dropna(axis=1)
# cna_ext_val = pd.read_csv('/mnt/disks/pancan/data/mel_dfci_2019/data_CNA.txt',
#                           delimiter='\t').set_index('Hugo_Symbol').T.dropna(axis=1)
# ext_val = pd.read_csv('/mnt/disks/pancan/data/mel_dfci_2019/data_clinical_sample.txt',
#                              delimiter='\t').set_index('Sample Identifier').iloc[4:]
# important_genes = list(pd.read_csv('/mnt/disks/pancan/m1000/cancer_genes.txt')['genes'].values)
# joint_genes = list(set(important_genes).intersection(list(rna_ext_val.columns), list(cna_ext_val.columns)))
# gene_list = random.sample(joint_genes, 500)
# random_genes_a = list(rna_ext_val.sample(5, axis=1).columns)
# random_genes_b = list(cna_ext_val.sample(5, axis=1).columns)
# joint_samples = list(rna_ext_val.sample(20).join(cna_ext_val, rsuffix='_cna', how='inner').index)
# random_samples_a = list(rna_ext_val.sample(5, axis=0).index)
# random_samples_b = list(cna_ext_val.sample(5, axis=0).index)
# random_samples_c = list(cna_ext_val.sample(5, axis=0).index)
# random_samples_d = list(cna_ext_val.sample(5, axis=0).index)
# test_rna = rna_ext_val.loc[joint_samples+random_samples_a][joint_genes+random_genes_a].copy().drop_duplicates()
# test_cna = cna_ext_val.loc[joint_samples+random_samples_b][joint_genes+random_genes_b].copy().drop_duplicates()
# test_add = ext_val.loc[joint_samples+random_samples_c][['Purity', 'Ploidy']].copy().drop_duplicates()
# test_y = ext_val.loc[joint_samples+random_samples_d][['Heterogeneity']].copy().drop_duplicates()
# test_rna.reset_index(inplace=True)
# test_cna.reset_index(inplace=True)
# test_add.reset_index(inplace=True)
# test_y.reset_index(inplace=True)
# test_rna.rename(columns={'index': 'sample_id'}, inplace=True)
# test_cna.rename(columns={'index': 'sample_id'}, inplace=True)
# test_add.rename(columns={'Sample Identifier': 'sample_id'}, inplace=True)
# test_y.rename(columns={'Sample Identifier': 'sample_id'}, inplace=True)
# test_rna.to_csv('../data/test_data/rna.csv', index=False)
# test_cna.to_csv('../data/test_data/cna.csv', index=False)
# test_add.to_csv('../data/test_data/add.csv', index=False)
# test_y.to_csv('../data/test_data/y.csv', index=False)
# with open('../data/test_data/gene_sublist.txt', 'wb') as fp:
#     pickle.dump(gene_list, fp)

Read test data

In [3]:
test_rna = pd.read_csv('../data/test_data/rna.csv').set_index('sample_id')
test_cna = pd.read_csv('../data/test_data/cna.csv').set_index('sample_id')
test_add = pd.read_csv('../data/test_data/add.csv').set_index('sample_id')
test_y = pd.read_csv('../data/test_data/y.csv').set_index('sample_id')

with open('../data/test_data/gene_sublist.txt', 'rb') as fp:
    gene_list = pickle.load(fp)

In [4]:
genetic_data = {'rna': test_rna, 'cna': test_cna}

In [5]:
train_dataset, test_dataset = pnet_loader.generate_train_test(genetic_data,
                                                              test_y, 
                                                              additional_data=test_add,
                                                              test_split=0.2,
                                                              gene_set=gene_list,
                                                              collinear_features=2)

Given 2 Input modalities
Found 20 overlapping indicies
Initializing Train Dataset
Found 500 overlapping genes
generated input DataFrame of size (16, 1000)
Initializing Test Dataset
Found 500 overlapping genes
generated input DataFrame of size (4, 1000)
Replace input of: PIK3R1_cna with collinear feature.
Replace input of: SDC4_cna with collinear feature.


In [6]:
# Debug: Let's check what's happening with the genes
print("Length of gene_list:", len(gene_list))
print("Length of train_dataset.genes:", len(train_dataset.genes))
print("Shape of train_dataset.input_df:", train_dataset.input_df.shape)
print("Number of columns in input_df:", len(train_dataset.input_df.columns))
print("\nFirst 5 genes from gene_list:", gene_list[:5])
print("First 5 genes from train_dataset.genes:", train_dataset.genes[:5])
print("First 5 columns from input_df:", list(train_dataset.input_df.columns)[:5])
print("\nLast 5 columns from input_df:", list(train_dataset.input_df.columns)[-5:])

Length of gene_list: 500
Length of train_dataset.genes: 500
Shape of train_dataset.input_df: (16, 1000)
Number of columns in input_df: 1000

First 5 genes from gene_list: ['PPFIBP1', 'NF1', 'CCDC6', 'PRDM2', 'RNF43']
First 5 genes from train_dataset.genes: ['MNX1', 'MYCN', 'FGFR3', 'ABL1', 'SF3B1']
First 5 columns from input_df: ['MNX1_rna', 'MYCN_rna', 'FGFR3_rna', 'ABL1_rna', 'SF3B1_rna']

Last 5 columns from input_df: ['GRM3_cna', 'KDSR_cna', 'FAS_cna', 'DAXX_cna', 'TP53_cna']


In [7]:
assert set(gene_list) == set(train_dataset.genes), 'Training dataset expected to have the same gene set as in file'

# Check that all genes from train_dataset.genes appear in the columns with appropriate suffixes
input_cols = list(train_dataset.input_df.columns)
genes_from_cols = set()
for col in input_cols:
    # Remove the modality suffix to get the gene name
    if col.endswith('_rna') or col.endswith('_cna'):
        gene = col.rsplit('_', 1)[0]
        genes_from_cols.add(gene)

assert set(train_dataset.genes) == genes_from_cols, 'Training data genes should match the genes in input_df columns'

# Since we have 2 modalities (rna and cna), we expect 1000 columns for 500 genes
assert train_dataset.input_df.shape == torch.Size([16, 1000]), 'Input DataFrame expected to be a of size\
                                                        [16, 1000], got: {}'.format(train_dataset.input_df.shape)
assert train_dataset.x.shape == torch.Size([16, 1000]), 'Small train dataset expected to be a tensor of size\
                                                        [16, 1000], got: {}'.format(train_dataset.x.shape)
assert train_dataset.y.shape == torch.Size([16, 1]), 'Small train dataset expected to be a tensor of size\
                                                        [16, 1], got: {}'.format(train_dataset.y.shape)

In [8]:
train_loader, val_loader = pnet_loader.to_dataloader(train_dataset, test_dataset, 64)

In [9]:
test_y_bin = test_y.apply(lambda x: round(2*x)).astype(int)

In [10]:
# Use the existing gene_list from test data as cancer genes for testing
# Note: This is a subset of 500 cancer genes used for testing purposes
canc_genes = gene_list
print(f"Using {len(canc_genes)} cancer genes from test data for analysis")

Using 500 cancer genes from test data for analysis


In [11]:
class_weights = util.get_class_weights(torch.tensor(test_y_bin.values).view(-1))
model, train_scores, test_scores, train_dataset, test_dataset = Pnet.run(genetic_data,
                                                                         test_y_bin,
                                                                         verbose=True,
                                                                         early_stopping=True,
                                                                         epochs=10,
                                                                         batch_size=10,
                                                                         loss_weight=class_weights,
                                                                         loss_fn=nn.BCEWithLogitsLoss(reduce=None),
                                                                         gene_set = canc_genes
                                                                        )

x_train = train_dataset.x
additional_train = train_dataset.additional
y_train = train_dataset.y
x_test = test_dataset.x
additional_test = test_dataset.additional
y_test = test_dataset.y

model.to('cpu')
pred, preds = model(x_test, additional_test)
y_pred_proba = model.predict_proba(x_test, additional_test).detach()
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
test_auc = metrics.roc_auc_score(y_test, y_pred_proba)
#create ROC curve
plt.plot(fpr,tpr, color="darkorange", label="ROC curve (area = %0.2f)" % test_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend(loc="lower right")
plt.show()

Task defined: BC 
 if this is not the intended task please specify task
Given 2 Input modalities
Found 20 overlapping indicies
Initializing Train Dataset
Found 500 overlapping genes
generated input DataFrame of size (16, 1000)
Initializing Test Dataset
Found 500 overlapping genes
generated input DataFrame of size (4, 1000)
Found 500 overlapping genes


/procedure/pnet/src/pnet/ReactomeNetwork.py:197: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gene_connections = pd.DataFrame(index=self.gene_list, columns=pathway_nodes).fillna(0)
/procedure/pnet/src/pnet/ReactomeNetwork.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pathway_connections = pd.DataFrame(index=pathway_nodes, columns=higher_level_pathway_nodes).fillna(0)
/procedure/pnet/src/pnet/ReactomeNetwork.py:204: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Curre

Epoch 1 of 10
Train Loss: 9.513814926147461
Test Loss: 12.612500190734863
Epoch 2 of 10
Train Loss: 8.882773399353027
Test Loss: 12.545892715454102
Epoch 3 of 10
Train Loss: 8.507103443145752
Test Loss: 12.465489387512207
Epoch 4 of 10
Train Loss: 7.777812242507935
Test Loss: 12.393580436706543
Epoch 5 of 10
Train Loss: 7.769348382949829
Test Loss: 12.313848495483398
Epoch 6 of 10
Train Loss: 7.991897344589233
Test Loss: 12.247538566589355
Epoch 7 of 10
Train Loss: 7.145643472671509
Test Loss: 12.155467987060547
Epoch 8 of 10
Train Loss: 6.987882137298584
Test Loss: 12.075577735900879
Epoch 9 of 10
Train Loss: 6.68674373626709
Test Loss: 11.988466262817383
Epoch 10 of 10
Train Loss: 6.607650518417358
Test Loss: 11.893295288085938


/root/miniconda3/envs/pnet/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:1133: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
model

In [ ]:
class_weights = util.get_class_weights(torch.tensor(test_y_bin.values).view(-1))
model, train_scores, test_scores, train_dataset, test_dataset = Pnet.run_geneset(genetic_data,
                                                                         test_y_bin,
                                                                        geneset_path='/mnt/disks/pancan/pnet/data/hallmark/c6.all.v2022.1.Hs.symbols.gmt',
                                                                         verbose=True,
                                                                         early_stopping=True,
                                                                         epochs=10,
                                                                         batch_size=10,
                                                                         loss_weight=class_weights,
                                                                         loss_fn=nn.BCEWithLogitsLoss(reduce=None),
                                                                         genes = canc_genes
                                                                        )

x_train = train_dataset.x
additional_train = train_dataset.additional
y_train = train_dataset.y
x_test = test_dataset.x
additional_test = test_dataset.additional
y_test = test_dataset.y

model.to('cpu')
pred, preds = model(x_test, additional_test)
y_pred_proba = model.predict_proba(x_test, additional_test).detach()
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
test_auc = metrics.roc_auc_score(y_test, y_pred_proba)
#create ROC curve
plt.plot(fpr,tpr, color="darkorange", label="ROC curve (area = %0.2f)" % test_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.legend(loc="lower right")
plt.show()

In [ ]:
model

In [ ]:
import GenesetNetwork
gn = GenesetNetwork.GenesetNetwork(canc_genes, '/mnt/disks/pancan/pnet/data/hallmark/c6.all.v2022.1.Hs.symbols.gmt')

In [ ]:
gn.pathway_encoding['ID'].nunique()

In [ ]:
[l.shape for l in gn.gene_layers]

In [ ]:
[l.shape for l in gn.pathway_layers]

In [ ]:
gn.is_no_bugs()

In [ ]:
gn.pathway_layers[0]

In [ ]:
gn.genes2pathways[gn.genes2pathways['gene'] == 'CCND2']

In [ ]:
gn.genes2pathways.index.value_counts()

In [ ]:
gn.genes2pathways['gene']

In [ ]:
pd.get_dummies(gn.genes2pathways['gene']).join(gn.genes2pathways['pathway']).groupby('pathway').sum().T

In [ ]:
df = pd.DataFrame(index=['train_loss', 'test_loss'], data=[train_scores, test_scores]).transpose()
df['auc'] = 0.75

In [ ]:
df

In [ ]:
task = util.get_task(test_y_bin)
target = util.format_target(test_y_bin, task)
train_dataset, test_dataset = pnet_loader.generate_train_test(genetic_data, target)
reactome_network = ReactomeNetwork.ReactomeNetwork(train_dataset.get_genes())

In [ ]:
model = Pnet.PNET_NN(reactome_network=reactome_network, task=task, nbr_gene_inputs=len(genetic_data),
                     loss_weight=class_weights, loss_fn=nn.BCEWithLogitsLoss(reduce=False))
train_loader, test_loader = pnet_loader.to_dataloader(train_dataset, test_dataset, 10)
model, train_scores, test_scores = Pnet.train(model, train_loader, test_loader, save_path='../results/model', epochs=10)

In [ ]:
def deeppathwayVAE_mask(gene_list, n_level=5):
    network = ReactomeNetworkOG.ReactomeNetwork(gene_list, n_levels=n_level)
    return network

In [ ]:
ReactomeNetwork = deeppathwayVAE_mask(gene_list, n_level=5)

In [ ]:
ReactomeNetwork.masks[-2].shape

In [ ]:
model.reactome_network.pathway_layers[0].shape

In [ ]:
# Already defined canc_genes above, but if needed again:
# canc_genes = gene_list
print(f"Cancer genes already loaded: {len(canc_genes)} genes")